# Extract

In [1]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine

from config import username, password, database_name, localhost_number

In [2]:
# Create filepaths to read
filepath_college_type = 'Resources/salaries-by-college-type.csv'
filepath_salaries = 'Resources/salaries-by-region.csv'

In [3]:
# Read CSV files and store as dataframes
college_type_df = pd.read_csv(filepath_college_type)
salaries_df = pd.read_csv(filepath_salaries)

# Transform

In [6]:
# Start with college_type data, drop unnecessary columns
college_type = college_type_df[['School Name','School Type']].copy()

# Rename columns (lower-case, no spaces)
college_type.rename(columns={'School Name':'school_name','School Type':'school_type'},inplace=True)

# Remove duplicate schools
college_type.drop_duplicates('school_name',inplace=True)

# Set school_name as index
college_type.set_index('school_name',inplace=True)

In [8]:
# Next with salaries data, drop unnecessary columns
salaries = salaries_df[['School Name','Region','Starting Median Salary','Mid-Career Median Salary']].copy()

# Rename columns (lower-case, no spaces)
salaries.rename(columns={'School Name':'school_name',
                         'Region':'region',
                         'Starting Median Salary':'starting_median_salary',
                        'Mid-Career Median Salary':'mid_career_median_salary'}, inplace=True)

# Clean up data type of salary data in two columns
salaries['starting_median_salary'] = salaries['starting_median_salary'].str.replace('$', '')
salaries['starting_median_salary'] = salaries['starting_median_salary'].str.replace(',', '')

salaries['mid_career_median_salary'] = salaries['mid_career_median_salary'].str.replace('$', '')
salaries['mid_career_median_salary'] = salaries['mid_career_median_salary'].str.replace(',', '')

salaries['starting_median_salary'] = salaries['starting_median_salary'].astype('float')
salaries['mid_career_median_salary'] = salaries['mid_career_median_salary'].astype('float')

# Remove duplicate schools
salaries.drop_duplicates('school_name',inplace=True)

# Set school_name as index
salaries.set_index('school_name',inplace=True)

# Load

In [9]:
# Create connection to local database
rds_connection_string = f"{username}:{password}@localhost:{localhost_number}/{database_name}"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [10]:
# Check for tables
engine.table_names()

['college_type', 'salaries']

In [11]:
# Load college_type dataframe into sql table using connection
college_type.to_sql(name='college_type', con=engine, if_exists='append', index=True)

In [12]:
# load salaries dataframe into sql table using connection
salaries.to_sql(name='salaries', con=engine, if_exists='append', index=True)